In [ ]:
features_selected = ['department', 'region', 'awards_won?', 
                     'previous_year_rating', 'KPIs_met >80%',
                      'avg_training_score', 'total_score', 'pca_combined']
                      

In [ ]:
features_selected_categorical = ['department', 'region', 'awards_won?','KPIs_met >80%']
features_selected_ordinal = ['previous_year_rating']
features_selected_numerical = ['avg_training_score', 'total_score', 'pca_combined']

In [ ]:
# Save to disk
# # X.loc[:, features_selected] -> to store only selected features
# X.to_csv(path_or_buf='/content/drive/My Drive/Colab Notebooks/data/HR_processed.csv', index=False)
# pd.DataFrame(data=train_index,columns=['train_index']).to_csv(
#     path_or_buf='/content/drive/My Drive/Colab Notebooks/data/HR_train_index.csv',
#     index=False)
# pd.DataFrame(data=validate_index,columns=['validate_index']).to_csv(
#     path_or_buf='/content/drive/My Drive/Colab Notebooks/data/HR_validate_index.csv',
#     index=False)

# Utility Methods for Modelling

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes, 
                          title=None, for_data='train'):
    '''
    for_data = Train or CV or Test
    '''
    
    if not title:
        title = 'Confusion Matrix'
    
    cm = confusion_matrix(y_true, y_pred)

    # Fliping is needed, because originally
    # True/Actual labels are on Y-Axis and 
    # Predicted labels are on X-Axis, 
    # but we need Predicted labels on Y-Axis
    # flip horizontally: axis = 1
    cm = np.flip(np.flip(cm, 0),1) #changed
    
    # Since we have flipped Actual & Predicted
    # We need to flip the class Labels aswell
    # the labels are in Lexicographical order in
    # data. Hence 0(Rejected) followed by 1(Accepted).
    classes = np.flip(classes)
    
    #classes = classes[unique_labels(y_true, y_pred)]

    if for_data.lower().strip() == 'test':
        cmap = plt.cm.Oranges
    else:
        cmap = plt.cm.Blues

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    
    #ax.figure.colorbar(im, ax=ax) # Equivalent to 'cbar' in sns.heatmap()
    
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[0]),# [1]
           yticks=np.arange(cm.shape[1]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Predicted label',
           xlabel='True label')# changed
    
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    
    # Loop over data dimensions and create text annotations.
    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[1]):
        for j in range(cm.shape[0]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    
    plt.show()
    


In [ ]:
def stratified_sample(y, sample_size):
    '''
        Parameters :
        ---------------------------------------
        y : Target label

        sample_size : Integer or Float value indicating 
                      number of samples to be drawn

        Return:
        --------------------------------
        list of index of samples drawn
    '''

    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    # check if sample size or sample fraction provided
    # numbers.Integral --> needed in case of Long integers
    if isinstance(sample_size, int):
        if sample_size > len(y):
            raise ValueError('sample_size should be less than or equal to length of y.')
    elif isinstance(sample_size, float):
        sample_size = math.ceil(len(y) * sample_size)
        int_flag = False
    else:
        raise ValueError('sample_size can only be integer or float')

    # no of instances of each label
    vc = y.value_counts()
    
    # count fraction of each label in original data
    vc = vc / len(y)
    
    # holds the index of the subsample drawn
    # this wil be returned
    sub_sample_index = []

    for label in vc.index:
        # calculate no of samples for each class to be returned
        count = math.ceil(sample_size * vc[label])

        # draw the sample (index)
        # list() => index data is not supported by random.sample()
        sub_sample = random.sample(list(y[y==label].index), count) 

        sub_sample_index.extend(sub_sample)

    
    return sub_sample_index


In [ ]:
   
def tuned_model(tuned_base_model, X, y):

    tuned_base_model = tuned_base_model.fit(X, y)

    calibrated_model = CalibratedClassifierCV(base_estimator=tuned_base_model, cv='prefit')
    calibrated_model = calibrated_model.fit(X, y)

    return tuned_base_model, calibrated_model